<a href="https://colab.research.google.com/github/jayantc14/News-Classification/blob/main/CNNFakenewsTensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd 
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import scale

import xgboost as xgb
from xgboost import XGBClassifier
from xgboost import plot_importance
import gc # for deleting unused variables
%matplotlib inline


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/IIM/data")

In [ ]:
df.drop(['Unnamed: 0'],axis = 1,inplace  = True)

In [ ]:
df

,Analytic,Authentic,Tone,WPS,Sixltr,number,quant,posemo,negemo,tentat,certain,achieve,power,reward,risk,focuspast,focuspresent,focusfuture,FREScore,target
0,97.02,22.27,35.07,23.41,28.44,2.80,1.34,1.60,1.07,1.20,0.27,1.20,5.87,0.80,0.93,4.01,6.81,2.00,42.04,1
1,96.86,13.98,49.52,20.23,32.85,2.39,1.28,2.71,1.44,0.96,0.48,0.96,8.61,0.80,1.12,3.83,5.10,2.07,32.57,1
2,94.73,16.26,29.43,24.05,27.35,0.88,1.75,1.09,0.88,3.94,0.66,0.66,6.13,0.44,0.00,5.47,5.47,1.97,39.91,1
3,96.40,12.12,55.62,22.29,32.19,2.11,0.53,2.11,0.53,1.32,0.26,1.32,6.60,0.53,0.00,6.33,3.43,0.53,45.49,1
4,96.29,22.73,15.88,22.05,27.21,4.19,3.14,1.16,1.86,0.93,0.81,0.81,3.60,0.70,0.93,3.49,5.47,0.23,50.80,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
44873,96.34,16.74,46.26,49.00,28.01,1.11,2.04,3.90,2.78,2.60,0.93,0.74,3.15,0.93,0.37,5.19,3.71,0.37,22.35,0
44874,94.03,14.89,56.75,27.73,19.02,3.28,1.97,2.95,1.31,1.97,0.33,2.30,5.57,2.30,0.33,3.93,6.23,0.98,51.41,0
44875,90.88,33.24,22.15,38.88,22.66,1.59,1.99,2.48,2.71,2.22,1.54,1.68,4.68,0.77,1.03,2.83,5.94,1.52,28.24,0
44876,91.99,25.14,49.16,47.70,20.34,4.40,2.31,2.52,1.26,1.05,2.10,2.10,3.14,1.68,1.05,3.77,7.13,0.84,27.16,0


In [ ]:
train, test = train_test_split(df, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

28721 train examples
7181 validation examples
8976 test examples


In [ ]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

In [ ]:
batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [ ]:
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  
  print('A batch of targets:', label_batch )

Every feature: ['Analytic', 'Authentic', 'Tone', 'WPS', 'Sixltr', 'number', 'quant', 'posemo', 'negemo', 'tentat', 'certain', 'achieve', 'power', 'reward', 'risk', 'focuspast', 'focuspresent', 'focusfuture', 'FREScore']
A batch of targets: tf.Tensor([0 0 1 1 0], shape=(5,), dtype=int64)


In [ ]:
# We will use this batch to demonstrate several types of feature columns
example_batch = next(iter(train_ds))[0]

In [ ]:
# A utility method to create a feature column
# and to transform a batch of data
def demo(feature_column):
  feature_layer = layers.DenseFeatures(feature_column)
  print(feature_layer(example_batch).numpy())

In [ ]:
photo_count = feature_column.numeric_column('quant')
demo(photo_count)

[[2.11]
 [1.29]
 [2.22]
 [2.24]
 [0.83]]


In [ ]:
feature_columns = []

# numeric cols
for header in ['Analytic', 'Authentic', 'Tone', 'WPS', 'Sixltr', 'number', 'quant', 'posemo', 'negemo', 'tentat', 'certain', 'achieve', 'power', 'reward', 'risk', 'focuspast', 'focuspresent', 'focusfuture', 'FREScore']:
  feature_columns.append(feature_column.numeric_column(header))

In [ ]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [ ]:
batch_size = 50
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [ ]:
model = tf.keras.Sequential([
  feature_layer,
  layers.Dense(128, activation='relu'),
  layers.Dense(128, activation='relu'),
  
  layers.Dropout(.1),
  layers.Dense(1)
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=500)

Epoch 1/500
Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.
563/575 [============================>.] - ETA: 0s - loss: 0.4573 - accuracy: 0.8293WARNING:tensorflow:Layers in a Sequential model should only have a single input tensor, but we receive a <class 'dict'> input: {'Analytic': <tf.Tensor 'ExpandDims:0' shape=(None, 1) dtype=float64>, 'Authentic': <tf.Tensor 'ExpandDims_1:0' shape=(None, 1) dtype=float64>, 'Tone': <tf.Tensor 'ExpandDims_4:0' shape=(None, 1) dtype=float64>, 'WPS': <tf.Tensor 'ExpandDims_5:0' shape=(None, 1) dtype=float64>, 'Sixltr': <tf.Tensor 'ExpandDims_3:0' shape=(None, 1) dtype=float64>, 'number': <tf.Tensor 'ExpandDims_12:0' shape=(None, 1) dtype=float64>, 'quant': <tf.Tensor 'ExpandDims_15:0' shape=(None, 1) dtype=float64>, 'posemo': <tf.Tensor 'ExpandDims_13:0' shape=(None, 1) dtype=float64>, 'negemo': <tf.Tensor 'ExpandDims_11:0' shape=(None, 1) dtype=float64>, 'tentat': <tf.Tensor 'ExpandDims_18

In [ ]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

180/180 [==============================] - 0s 3ms/step - loss: 0.6646 - accuracy: 0.9151
Accuracy 0.9151069521903992
